## データ型を確認する

In [10]:
import pandas as pd
import numpy as np

# トレインデータの読み込み
data_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test.csv'
test_data = pd.read_csv(data_path, low_memory=False)

# データ型の確認
print("Data types of each column:\n", test_data.dtypes)

print(test_data.info())

Data types of each column:
 Age           int64
Gender       object
T_Bil       float64
D_Bil       float64
ALP         float64
ALT_GPT     float64
AST_GOT     float64
TP          float64
Alb         float64
AG_ratio    float64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       350 non-null    int64  
 1   Gender    350 non-null    object 
 2   T_Bil     350 non-null    float64
 3   D_Bil     350 non-null    float64
 4   ALP       350 non-null    float64
 5   ALT_GPT   350 non-null    float64
 6   AST_GOT   350 non-null    float64
 7   TP        350 non-null    float64
 8   Alb       350 non-null    float64
 9   AG_ratio  350 non-null    float64
dtypes: float64(8), int64(1), object(1)
memory usage: 27.5+ KB
None


### 特徴量の分布を確認して対数変換を適用する

In [39]:
import numpy as np
import pandas as pd

# データファイルを読み込み
data_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test.csv'
test_data = pd.read_csv(data_path, low_memory=False)

# トレーニングデータと同様に対数変換を適用
test_data['T_Bil_log'] = np.log1p(test_data['T_Bil'])
test_data['D_Bil_log'] = np.log1p(test_data['D_Bil'])
test_data['ALP_log'] = np.log1p(test_data['ALP'])
test_data['ALT_GPT_log'] = np.log1p(test_data['ALT_GPT'])
test_data['AST_GOT_log'] = np.log1p(test_data['AST_GOT'])

# 0以下の値がないことが確認されている場合np.logを適用する
test_data['TP_log'] = np.log(test_data['TP'])
test_data['Alb_log'] = np.log(test_data['Alb'])
test_data['AG_ratio_log'] = np.log(test_data['AG_ratio'])

# データをファイルに保存
save_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_log.csv'
test_data.to_csv(save_path, index=False)
print(f"対数変換後のデータが {save_path} に保存されました。")

対数変換後のデータが /Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_log.csv に保存されました。


### 対数変換前の特徴量を削除する

In [40]:
import pandas as pd

# トレインデータの読み込み
data_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_log.csv'
test_data = pd.read_csv(data_path, low_memory=False)

# '不要な列を削除して他の特徴量を残す
test_data = test_data.drop(columns=['T_Bil', 'D_Bil', 'ALP', 'ALT_GPT', 'AST_GOT',
                                      'TP', 'Alb', 'AG_ratio'])

# 新しいデータセットの保存
save_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_drop_original.csv'
test_data.to_csv(save_path, index=False)

print(test_data.dtypes)
print(f"データが {save_path} に保存されました。")

Age               int64
Gender           object
T_Bil_log       float64
D_Bil_log       float64
ALP_log         float64
ALT_GPT_log     float64
AST_GOT_log     float64
TP_log          float64
Alb_log         float64
AG_ratio_log    float64
dtype: object
データが /Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_drop_original.csv に保存されました。


## 新しい特徴量を作成する
トレーニングデータで作成した特徴量と同じ特徴量を作成します。

In [1]:
import numpy as np
import pandas as pd
import os

# データファイルを読み込み
data_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_drop_original.csv'
test_data = pd.read_csv(data_path, low_memory=False)


# 年齢をカテゴリに分ける
# test_data['Age_bucket'] = pd.cut(test_data['Age'], bins=[0, 20, 40, 60, 80, 100], labels=[1, 2, 3, 4, 5])

# 新しい特徴量を作成する
# グロブリンを計算
# test_data['Globulin'] = test_data['TP_log'] - test_data['Alb_log']

# 仮定的な肝機能スコア
# test_data['Liver_Function_Combined_Score'] = (
    # test_data['AST_GOT_log'] + 
    # test_data['T_Bil_log'] - 
    # test_data['Alb_log']
# )

# 比を作成する場合
test_data['TP_Alb_ratio'] = test_data['TP_log'] / (test_data['Alb_log'] + 1e-8)  # 0除算を避けるために微小値を加える

# ディレクトリが存在するか確認し、存在しない場合は作成
save_dir = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# データを保存
test_data_path = os.path.join(save_dir, 'test_new_features.csv')
test_data.to_csv(test_data_path, index=False)

print(f"新しい特徴量を作成しデータを保存しました: {test_data_path}")

新しい特徴量を作成しデータを保存しました: /Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_new_features.csv


### 相関関係の高すぎる特徴量を削除する

In [2]:
import pandas as pd

# トレインデータの読み込み
data_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_new_features.csv'
test_data = pd.read_csv(data_path, low_memory=False)

# '不要な列を削除して他の特徴量を残す
test_data = test_data.drop(columns=['TP_log', 'Alb_log'])

# 新しいデータセットの保存
save_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_drop_dupe.csv'
test_data.to_csv(save_path, index=False)

print(test_data.dtypes)
print(f"データが {save_path} に保存されました。")

Age               int64
Gender           object
T_Bil_log       float64
D_Bil_log       float64
ALP_log         float64
ALT_GPT_log     float64
AST_GOT_log     float64
AG_ratio_log    float64
TP_Alb_ratio    float64
dtype: object
データが /Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_drop_dupe.csv に保存されました。


## ワンホットエンコーディングを適用する

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import joblib

# testデータの読み込み
data_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_drop_dupe.csv'
test_data = pd.read_csv(data_path, low_memory=False)

# ワンホットエンコーダをロードする
onehot_encoder_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/train/onehot_encoder.pkl'
onehot_encoder = joblib.load(onehot_encoder_path)

# ワンホットエンコーディングを適用するカテゴリ変数
categorical_columns = ['Gender']

# カテゴリ変数がテストデータに存在するか確認
for column in categorical_columns:
    if column not in test_data.columns:
        raise ValueError(f"テストデータに '{column}' 列が存在しません。")

# ワンホットエンコーディングを適用
encoded_categorical_data = onehot_encoder.transform(test_data[categorical_columns])

# データフレームに変換
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=onehot_encoder.get_feature_names_out(categorical_columns))

# 元のデータフレームにエンコードしたデータを結合
test_data = test_data.drop(columns=categorical_columns)
test_data = pd.concat([test_data.reset_index(drop=True), encoded_categorical_df.reset_index(drop=True)], axis=1)

# データの確認
print(test_data.head())

# 新しい特徴量を含むデータを保存
test_encoded_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_onehot_encoded.csv'
test_data.to_csv(test_encoded_path, index=False)
print(f"エンコードされたデータを保存しました: {test_encoded_path}")

   Age  T_Bil_log  D_Bil_log   ALP_log  ALT_GPT_log  AST_GOT_log  \
0   25   0.588785   0.171260  5.372723     2.788740     3.020261   
1   26   0.606967   0.110133  5.460168     2.443143     2.875445   
2   64   0.583238   0.180603  5.380081     2.745879     3.077122   
3   65   0.606501   0.161899  5.307778     2.626143     3.049392   
4   34   0.524904   0.127076  5.345706     3.057722     3.261808   

   AG_ratio_log  TP_Alb_ratio  Gender_Female  Gender_Male  
0      0.253095      1.498312            0.0          1.0  
1      0.030105      1.688389            1.0          0.0  
2      0.245749      1.566148            0.0          1.0  
3      0.243001      1.491218            0.0          1.0  
4      0.246921      1.511889            1.0          0.0  
エンコードされたデータを保存しました: /Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_onehot_encoded.csv


### 冗長な特徴量(Gender_Female)を削除する

In [5]:
import pandas as pd

# トレインデータの読み込み
data_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_onehot_encoded.csv'
test_data = pd.read_csv(data_path, low_memory=False)

# '不要な列を削除して他の特徴量を残す
test_data = test_data.drop(columns=['Gender_Female'])

# 新しいデータセットの保存
save_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/train_drop_GenderFemale.csv'
test_data.to_csv(save_path, index=False)

print(test_data.dtypes)
print(f"データが {save_path} に保存されました。")


Age               int64
T_Bil_log       float64
D_Bil_log       float64
ALP_log         float64
ALT_GPT_log     float64
AST_GOT_log     float64
AG_ratio_log    float64
TP_Alb_ratio    float64
Gender_Male     float64
dtype: object
データが /Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/train_drop_GenderFemale.csv に保存されました。


##  RobustScalerでスケーリングを適用する

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import joblib

# テストデータの読み込み
data_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/train_drop_GenderFemale.csv'
test_data = pd.read_csv(data_path, low_memory=False)

# スケーリングを行うカラムを指定（これまでのcolumns_to_scaleを使用）
columns_to_scale = [ 
    'Age', 'T_Bil_log', 'D_Bil_log', 'ALP_log', 'ALT_GPT_log', 'AST_GOT_log', 
    'AG_ratio_log', 'TP_Alb_ratio'  
]

# スケーラーをロードしてスケーリング
scaler_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/train/train_robust_scaler.pkl'
scaler = joblib.load(scaler_path)
test_data[columns_to_scale] = scaler.transform(test_data[columns_to_scale])

# スケーリング後のデータを保存
output_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_scaled_robust_0919_1.csv'
test_data.to_csv(output_path, index=False)
print(f"スケーリング済みのデータを保存しました: {output_path}")


スケーリング済みのデータを保存しました: /Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_scaled_robust_0919_1.csv


In [1]:
import pandas as pd
import numpy as np
import joblib

# 実際のテストデータの読み込み
test_data_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_scaled_robust_0915_1.csv'
test_data = pd.read_csv(test_data_path, low_memory=False)

# 特徴量を選択
selected_features = [
    'T_Bil_log', 'AST_GOT_log', 'ALT_GPT_log', 'AG_ratio_log', 'D_Bil_log', 
    'ALP_log', 'TP_log', 'Alb_log', 'Age', 'Gender_Male'
]

# PCAモデルをロード
pca_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/model/pca_model.joblib'
pca = joblib.load(pca_path)

# PCAをテストデータに適用
test_data_pca = pca.transform(test_data[selected_features])

# PCA後のデータをDataFrameに変換
test_data_pca_df = pd.DataFrame(test_data_pca, columns=[f'PC{i+1}' for i in range(test_data_pca.shape[1])])

# PCA後のデータを保存
output_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_pca_transformed_data.csv'
test_data_pca_df.to_csv(output_path, index=False)

print(f"PCA適用後のテストデータを保存しました: {output_path}")



PCA適用後のテストデータを保存しました: /Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_pca_transformed_data.csv


# StandardScalerでスケーリングする

In [8]:
# import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

# テストデータの読み込み
data_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/train_drop_GenderFemale.csv'
test_data = pd.read_csv(data_path, low_memory=False)

# スケーリングを行うカラムを指定
columns_to_scale = [ 
    'Age', 'T_Bil_log', 'D_Bil_log', 'ALP_log', 'ALT_GPT_log', 'AST_GOT_log', 
    'AG_ratio_log', 'TP_Alb_ratio'  
]

# 無限大の値や非常に大きな値の処理を各カラムで個別に行う
for col in columns_to_scale:
    test_data[col] = test_data[col].replace([np.inf, -np.inf], np.nan)

# NaN値を平均で補完
for col in columns_to_scale:
    test_data[col] = test_data[col].fillna(test_data[col].mean())

# スケーラーをロードしてスケーリング
scaler_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/train/train_std_scaler.pkl'
scaler = joblib.load(scaler_path)
test_data[columns_to_scale] = scaler.transform(test_data[columns_to_scale])

# スケーリング後のデータを保存
output_path = '/Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_std_scaled_0919_1.csv'
test_data.to_csv(output_path, index=False)
print(f"スケーリング済みのデータを保存しました: {output_path}")


スケーリング済みのデータを保存しました: /Users/hayakawakazue/Downloads/signate/LiverDiseaseDetection/test/test_std_scaled_0919_1.csv
